# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st

from scipy.stats import linregress

from api_keys import weather_api_key
from citipy import citipy

ImportError: cannot import name 'animation' from partially initialized module 'matplotlib' (most likely due to a circular import) (C:\Users\tyler\anaconda3\envs\PythonData\lib\site-packages\matplotlib\__init__.py)

## Generate Cities List

In [ ]:
#lat & lng ranges
lat_range=(-90, 90)
lng_range=(-180,180)

#lists to hold location and cities
lat_lngs = []
cities = []

#random city selection
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

#nearest city id for random selection
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #add to list if unique name
    if city not in cities:
        cities.append(city)

len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#OpenWeatherMap url + api key & modifications needed for call
weather_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = weather_url + "appid=" + weather_api_key + "&units=" + units + "&q="

#print test city 
test = requests.get(f'{query_url}indianapolis').json()
display(test)

In [ ]:
# lists to hold data for df creation 
country = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
date = []

city_list = []

#run loop & print data request for all cities from random list
counter = 0

print('Beginning Data Retrieval')
print('------------')

for city in cities:
    
    response=requests.get(query_url + city).json()
    
    try:
        country.append(response['sys']['country'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        date.append(response['dt'])
        
        city_list.append(city)
        
        counter += 1
        
        print(f"Processing Records {counter} | {city}")
        
    except:
        print("City not found. Skipping.")
        pass
    
print('------------')
print('Data Retrieval Complete')
print('------------')

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#create df
city_df = pd.DataFrame({'City': city_list,
                       'Country': country,
                       'Lat': lat,
                       'Lng': lng,
                       'Max Temp': max_temp,
                       'Humidity': humidity,
                       'Cloudiness': cloudiness,
                       'Wind Speed': wind_speed,
                       'Date': date})
display(city_df)

#export df to csv
output_df = "../output_data/cities.csv"
city_df.to_csv(output_df, encoding="utf-8", index=False)

In [ ]:
city_df.describe()

## Inspect the data and remove the cities where the humidity > 100%.
----
Max humidity is at 100%. No outliers that need to be removed. 

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
x = city_df['Lat']
y = city_df['Max Temp']

plt.figure(1)
plt.scatter(x, y, c=y, cmap='Greens', edgecolor='gray')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.grid()
plt.title(f'City Latitude vs. Temperature (F) ({date})')

plt.savefig = "../output_data/Lat_v_Temp.png"
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
x_values = city_df['Lat']
y_values = city_df['Humidity']

plt.scatter(x_values, y_values, cmap='hot', edgecolor='gray')
plt.title(f'City Latitude vs. Humidity ({date})')
plt.grid()
plt.xlabel('Latitude')
plt.ylabel('Humidity')

plt.savefig = "../output_data/Lat_v_Humidity.png"
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
x_values = city_df['Lat']
y_values = city_df['Cloudiness']
plt.scatter(x_values, y_values, cmap='Purples', edgecolor='gray')
plt.title(f'City Latitude vs. Cloudiness (%) ({date})')
plt.grid()
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')

plt.savefig = "../output_data/Lat_v_Cloudiness.png"
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
x_values = city_df['Lat']
y_values = city_df['Wind Speed']

plt.scatter(x_values, y_values, cmap='BuPu', edgecolor='gray')
plt.title(f'City Latitude vs. Wind Speed (%) ({date})')
plt.grid()
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (%)')

plt.savefig = "../output_data/Lat_v_WindSpeed.png"
plt.show()

## Linear Regression

In [ ]:
def regressplot(x, y):
    correlation = st.pearsonr(x,y)
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x,y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x,y)
    plt.plot(x,regress_values,'r-')
    return line_eq

In [ ]:
north = city_df.loc[city_df['Lat'] >= 0]
south = city_df.loc[city_df['Lat'] < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
line_eq = regressplot(x=north['Lat'], y=north['Max Temp'])
plt.annotate(line_eq, (5,40), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')

plt.savefig = "../output_data/North_Hem_Temp.png"
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=south['Lat'], y=south['Max Temp'])
plt.annotate(linear_regress, (-55,80), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')

plt.savefig = "../output_data/South_Hem_Temp.png"
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=north['Lat'], y=north['Humidity'])
plt.annotate(linear_regress, (5,20), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')

plt.savefig = "../output_data/North_Humidity.png"
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=south['Lat'], y=south['Humidity'])
plt.annotate(linear_regress, (-55,30), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')

plt.savefig = "../output_data/South_Humidity.png"
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=north['Lat'], y=north['Cloudiness'])
plt.annotate(linear_regress, (5,10), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')

plt.savefig = "../output_data/North_Cloud.png"
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=south['Lat'], y=south['Cloudiness'])
plt.annotate(linear_regress, (-55,60), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')

plt.savefig = "../output_data/South_Cloud.png"
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=north['Lat'], y=north['Wind Speed'])
plt.annotate(linear_regress, (45,25), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')

plt.savefig = "../output_data/North_Wind.png"
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
linear_regress = regressplot(x=south['Lat'], y=south['Wind Speed'])
plt.annotate(linear_regress, (-25,24), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')

plt.savefig = "../output_data/South_Wind.png"
plt.show()